In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import numpy as np

In [4]:
def loadData(img_path, label_path):
    img_np = np.load(img_path)["arr_0"]
    img_np = img_np/255
    img_np = img_np.reshape(len(img_np), -1, 28, 28)
    label_np = np.load(label_path)["arr_0"]
    return torch.utils.data.TensorDataset(torch.from_numpy(img_np),
                                            torch.from_numpy(label_np))

train_data = loadData("kmnist/k49-train-imgs.npz", "kmnist/k49-train-labels.npz")
train_loader = torch.utils.data.DataLoader(train_data, batch_size=100, shuffle=True)
test_data = loadData("kmnist/k49-test-imgs.npz", "kmnist/k49-test-labels.npz")
test_loader = torch.utils.data.DataLoader(test_data)

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3) 
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout2d()
        self.fc1 = nn.Linear(12*12*64, 128)
        self.dropout2 = nn.Dropout2d()
        self.fc2 = nn.Linear(128, 49)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout1(x)
        x = x.view(-1, 12*12*64)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

net = Net().to("cuda")
model_dir = None #"./model_cnn"
print(net)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout2d(p=0.5)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (dropout2): Dropout2d(p=0.5)
  (fc2): Linear(in_features=128, out_features=49, bias=True)
)


In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [14]:
def train(epoch):
    net.train()
    for step, (data, target) in enumerate(train_loader, 0):
        data = data.to("cuda", dtype=torch.float)
        target = target.to("cuda", dtype=torch.long)
        optimizer.zero_grad()
        output = net(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if step == 0:
            print("Epoch:{}, Step:{}, Loss:{:.4}".format(epoch+1, step+1, loss.item()))

In [15]:
for epoch in range(100):
    train(epoch)

torch.save(net.state_dict(), "./model_cnn")

Epoch:1, Step:1, Loss:3.895
Epoch:2, Step:1, Loss:1.527
Epoch:3, Step:1, Loss:1.351
Epoch:4, Step:1, Loss:1.268
Epoch:5, Step:1, Loss:1.092
Epoch:6, Step:1, Loss:1.035
Epoch:7, Step:1, Loss:1.066
Epoch:8, Step:1, Loss:0.885
Epoch:9, Step:1, Loss:0.8737
Epoch:10, Step:1, Loss:0.8992
Epoch:11, Step:1, Loss:1.064
Epoch:12, Step:1, Loss:0.9152
Epoch:13, Step:1, Loss:0.5742
Epoch:14, Step:1, Loss:0.9035
Epoch:15, Step:1, Loss:0.6928
Epoch:16, Step:1, Loss:0.6677
Epoch:17, Step:1, Loss:0.637
Epoch:18, Step:1, Loss:0.6966
Epoch:19, Step:1, Loss:0.6408
Epoch:20, Step:1, Loss:0.7097
Epoch:21, Step:1, Loss:0.624
Epoch:22, Step:1, Loss:0.645
Epoch:23, Step:1, Loss:0.7458
Epoch:24, Step:1, Loss:0.4632
Epoch:25, Step:1, Loss:0.9269
Epoch:26, Step:1, Loss:0.5988
Epoch:27, Step:1, Loss:0.7619
Epoch:28, Step:1, Loss:0.7135
Epoch:29, Step:1, Loss:0.5457
Epoch:30, Step:1, Loss:0.6044
Epoch:31, Step:1, Loss:0.2729
Epoch:32, Step:1, Loss:0.4447
Epoch:33, Step:1, Loss:0.4911
Epoch:34, Step:1, Loss:0.4637
E

In [7]:
def test():
    net.eval()
    test_loss = 0
    correct = 0
    test_len = len(test_loader.dataset)
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to("cuda", dtype=torch.float)
            target = target.to("cuda", dtype=torch.long)
            output = net(data)
            test_loss += criterion(output, target).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= test_len
    print("Test: LossAverage:{:.4}, CorrectRate:{:.2%}({}/{})\n".format(test_loss, correct/test_len, correct, test_len))

test()

Test: LossAverage:0.4106, CorrectRate:89.49%(34495/38547)

